<a href="https://colab.research.google.com/github/zuzanna-w17/ML-course/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn

# Biblioteki

In [2]:
import numpy as np
import pandas as pd 
import sklearn

sklearn.__version__

'0.22.2.post1'

# Generacja danych

In [3]:
data = {
    "size": ["XL", "L", "M", "L", "M"],
    "color": ["red", "green", "blue", "green", "red"],
    "gender": ["female", "male", "male", "female", "female"],
    "price": [199.0, 89.0, 99.0, 129.0, 79.0],
    "weight": [500, 450, 300, 380, 410],
    "bought": ["yes", "no", "yes", "no", "yes"]
}

df_raw = pd.DataFrame(data=data)
df_raw

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


In [4]:
df = df_raw.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   size    5 non-null      object 
 1   color   5 non-null      object 
 2   gender  5 non-null      object 
 3   price   5 non-null      float64
 4   weight  5 non-null      int64  
 5   bought  5 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 368.0+ bytes


In [6]:
# change type "object" of categorical variables
for col in ["size", "color", "gender", "bought"]:
  df[col] = df[col].astype("category")

df["weight"] = df["weight"].astype("float")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   size    5 non-null      category
 1   color   5 non-null      category
 2   gender  5 non-null      category
 3   price   5 non-null      float64 
 4   weight  5 non-null      float64 
 5   bought  5 non-null      category
dtypes: category(4), float64(2)
memory usage: 628.0 bytes


In [8]:
# describe shows results only for numerical variables
df.describe()
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,5.0,119.0,48.476799,79.0,89.0,99.0,129.0,199.0
weight,5.0,408.0,75.299402,300.0,380.0,410.0,450.0,500.0


In [10]:
# show statistics of all variables
df.describe(include=["category"]).T

,count,unique,top,freq
size,5,3,M,2
color,5,3,red,2
gender,5,2,female,3
bought,5,2,yes,3


# Kodowanie danych

In [12]:
from sklearn.preprocessing import LabelEncoder # class to be used only for dependent variable!!!

le = LabelEncoder()
le.fit(df["bought"])
le.transform(df["bought"])

array([1, 0, 1, 0, 1])

In [13]:
# faster
le.fit_transform(df["bought"])

array([1, 0, 1, 0, 1])

In [18]:
# apply changes
df["bought"] = le.fit_transform(df["bought"])
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,1
1,L,green,male,89.0,450.0,0
2,M,blue,male,99.0,300.0,1
3,L,green,female,129.0,380.0,0
4,M,red,female,79.0,410.0,1


In [23]:
# inverse mapping
le.inverse_transform(df["bought"])

array([1, 0, 1, 0, 1])

In [22]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500.0,1
1,L,green,male,89.0,450.0,0
2,M,blue,male,99.0,300.0,1
3,L,green,female,129.0,380.0,0
4,M,red,female,79.0,410.0,1


In [24]:
from sklearn.preprocessing import OneHotEncoder # class to be used for independent variable!!!

encoder = OneHotEncoder(sparse=False)
encoder.fit(df[["size"]])

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=False)

In [25]:
encoder.transform(df[["size"]])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [26]:
encoder.categories_ # values: L, M XL in the above matrix's columns

[array(['L', 'M', 'XL'], dtype=object)]

In [27]:
# linear dependence to be removed (one column to be removed)
encoder = OneHotEncoder(drop="first", sparse=False)

encoder.fit(df[["size"]])
encoder.transform(df[["size"]])

array([[0., 1.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [1., 0.]])

In [28]:
# get_dummies in Pandas - alternative to teh above example
pd.get_dummies(data=df)

,price,weight,bought,size_L,size_M,size_XL,color_blue,color_green,color_red,gender_female,gender_male
0,199.0,500.0,1,0,0,1,0,0,1,1,0
1,89.0,450.0,0,1,0,0,0,1,0,0,1
2,99.0,300.0,1,0,1,0,1,0,0,0,1
3,129.0,380.0,0,1,0,0,0,1,0,1,0
4,79.0,410.0,1,0,1,0,0,0,1,1,0


In [29]:
# with linearity dependence removed
pd.get_dummies(data=df, drop_first=True)

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_male
0,199.0,500.0,1,0,1,0,1,0
1,89.0,450.0,0,0,0,1,0,1
2,99.0,300.0,1,1,0,0,0,1
3,129.0,380.0,0,0,0,1,0,0
4,79.0,410.0,1,1,0,0,1,0


In [30]:
# update new colnames
pd.get_dummies(data=df, drop_first=True, prefix="new")

,price,weight,bought,new_M,new_XL,new_green,new_red,new_male
0,199.0,500.0,1,0,1,0,1,0
1,89.0,450.0,0,0,0,1,0,1
2,99.0,300.0,1,1,0,0,0,1
3,129.0,380.0,0,0,0,1,0,0
4,79.0,410.0,1,1,0,0,1,0


In [31]:
# update new colnames
pd.get_dummies(data=df, drop_first=True, prefix_sep="-")

,price,weight,bought,size-M,size-XL,color-green,color-red,gender-male
0,199.0,500.0,1,0,1,0,1,0
1,89.0,450.0,0,0,0,1,0,1
2,99.0,300.0,1,1,0,0,0,1
3,129.0,380.0,0,0,0,1,0,0
4,79.0,410.0,1,1,0,0,1,0


In [32]:
# only selected columns to be coded
pd.get_dummies(data=df, drop_first=True, columns=["size"])

,color,gender,price,weight,bought,size_M,size_XL
0,red,female,199.0,500.0,1,0,1
1,green,male,89.0,450.0,0,0,0
2,blue,male,99.0,300.0,1,1,0
3,green,female,129.0,380.0,0,0,0
4,red,female,79.0,410.0,1,1,0


# Standaryzacja danych

In [36]:
df["price"]

0    199.0
1     89.0
2     99.0
3    129.0
4     79.0
Name: price, dtype: float64

In [35]:
df["price"].mean()

119.0

In [37]:
df["price"].std()

48.47679857416329

In [39]:
# standarized
(df["price"] - df["price"].mean()) / df["price"].std()

0    1.650274
1   -0.618853
2   -0.412568
3    0.206284
4   -0.825137
Name: price, dtype: float64

In [40]:
def standarize(x):
  return (x - x.mean()) / x.std()

standarize(df["price"])

0    1.650274
1   -0.618853
2   -0.412568
3    0.206284
4   -0.825137
Name: price, dtype: float64

In [42]:
# inbuilt library
from sklearn.preprocessing import scale

# quite different values as estimator can be implemented as biased or unbiased variance estimator
scale(df["price"])

array([ 1.84506242, -0.69189841, -0.4612656 ,  0.2306328 , -0.92253121])

In [43]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df[["price"]])
scaler.transform(df[["price"]])

array([[ 1.84506242],
       [-0.69189841],
       [-0.4612656 ],
       [ 0.2306328 ],
       [-0.92253121]])

In [44]:
scaler_price = StandardScaler()
df["price"] = scaler_price.fit_transform(df[["price"]])

scaler_weight = StandardScaler()
df["weight"] = scaler_weight.fit_transform(df[["weight"]])

In [45]:
df

,size,color,gender,price,weight,bought
0,XL,red,female,1.845062,1.366002,1
1,L,green,male,-0.691898,0.623610,0
2,M,blue,male,-0.461266,-1.603567,1
3,L,green,female,0.230633,-0.415740,0
4,M,red,female,-0.922531,0.029696,1


# Przygotowanie danych do modelu

In [46]:
df = df_raw.copy()
df

,size,color,gender,price,weight,bought
0,XL,red,female,199.0,500,yes
1,L,green,male,89.0,450,no
2,M,blue,male,99.0,300,yes
3,L,green,female,129.0,380,no
4,M,red,female,79.0,410,yes


In [47]:
le = LabelEncoder()

df["bought"] = le.fit_transform(df["bought"])

scaler = StandardScaler()
df[["price", "weight"]] = scaler.fit_transform(df[["price", "weight"]])

df = pd.get_dummies(data=df, drop_first=True)
df

,price,weight,bought,size_M,size_XL,color_green,color_red,gender_male
0,1.845062,1.366002,1,0,1,0,1,0
1,-0.691898,0.623610,0,0,0,1,0,1
2,-0.461266,-1.603567,1,1,0,0,0,1
3,0.230633,-0.415740,0,0,0,1,0,0
4,-0.922531,0.029696,1,1,0,0,1,0
